<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Understanding-Sales" data-toc-modified-id="Understanding-Sales-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Understanding Sales</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Data-structure" data-toc-modified-id="Data-structure-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Data structure</a></span></li><li><span><a href="#Limitations" data-toc-modified-id="Limitations-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Limitations</a></span></li><li><span><a href="#View-of-All-Data" data-toc-modified-id="View-of-All-Data-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>View of All Data</a></span></li><li><span><a href="#Summary-Stats" data-toc-modified-id="Summary-Stats-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Summary Stats</a></span></li></ul></li><li><span><a href="#Correlations-Between-Health-&amp;-Operations" data-toc-modified-id="Correlations-Between-Health-&amp;-Operations-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Correlations Between Health &amp; Operations</a></span><ul class="toc-item"><li><span><a href="#KPIs" data-toc-modified-id="KPIs-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>KPIs</a></span></li><li><span><a href="#Diseases/Cases" data-toc-modified-id="Diseases/Cases-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Diseases/Cases</a></span></li><li><span><a href="#Sales-Activity" data-toc-modified-id="Sales-Activity-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Sales Activity</a></span><ul class="toc-item"><li><span><a href="#Meets-Sales-Target" data-toc-modified-id="Meets-Sales-Target-1.2.3.1"><span class="toc-item-num">1.2.3.1&nbsp;&nbsp;</span>Meets Sales Target</a></span></li><li><span><a href="#Misses-Sales-Target" data-toc-modified-id="Misses-Sales-Target-1.2.3.2"><span class="toc-item-num">1.2.3.2&nbsp;&nbsp;</span>Misses Sales Target</a></span></li><li><span><a href="#Regressions" data-toc-modified-id="Regressions-1.2.3.3"><span class="toc-item-num">1.2.3.3&nbsp;&nbsp;</span>Regressions</a></span></li></ul></li></ul></li></ul></li></ul></div>

In [ ]:
## Setup
# load libs/modules
%run "../utils/load_envz.ipynb"

In [ ]:
# Load data set 
DATA_DIR = "../dataset"
DATA_FILE = "targeted_chv_support_ke-20180320.csv" 
dataset = pd.read_csv( "%s/%s" % (DATA_DIR, DATA_FILE) )

dataset.drop( "Phone Number", axis=1, inplace=True )

In [ ]:
## themes | variable - outcome mapping
argz_Units = ["CHP ID", "Households Registered"] 
argz_Ops = [ "Sales", "Community Events"]
argz_pregoz = [ "Pregnancy", "PNC Visit Rate", "PNC Visit Missed - #"]
argz_U5_care = ["Total U5 Assessments", "Total U5 Treatments"]
argz_U1_care = ["Total U1 Assessments", "Total U1 Treatments"]
argz_malaria = [ "All Malaria Assessments", "All Malaria Treatments", "Act 6pack Issue", "mRDT Used", "%mRDT"]
argz_U5_diseases = ["U5 Malaria", "U5 Diarrhea", "U5 Pneumonia"]

# "Total U1 Assessments",  assumes U1 follows U5 
argz_EPMs = ["Households Registered", "Sales", "Community Events", "Total Child Assessments", "Pregnancy"]

## Additional Columns
dataset["Country"] = "KE"
dataset["Total Child Assessments"] = dataset["Total U1 Assessments"] + dataset["Total U5 Assessments"]
dataset["Total Child Treatments"] = dataset["Total U1 Treatments"] + dataset["Total U5 Treatments"]
#dataset["HHQuintiles"] = pd.qcut( dataset["Households Registered"], 5, labels=["HH_Quint_1", "HH_Quint_2", "HH_Quint_3", "HH_Quint_4", "HH_Quint_5"])
dnul = pd.Series( ["Households Registered"]).apply( lambda x: quintilize(dataset, x, labelzPrefix="HH_Quints" ) )
dataset["Class_Sales_Levels"] = np.where( dataset["Sales"] < 2040 , "Below_2040", 
                                         np.where(dataset["Sales"] < 4080, "2040_to_4080", 
                                                 np.where(dataset["Sales"] < 6120, "4080_to_6120", 
                                                         np.where(dataset["Sales"] < 8160, "6120_to_8160", 
                                                                 np.where(dataset["Sales"] < 10200, "8160_to_10200", "Above_10K")))) ) 


# Understanding Sales

## Introduction

- Looking at KE CHP data for last 90 days as at 20-Mar-2018


- Some questions of interest
    - What makes sales better?
    - What relationships between sales and health activities? 

- The presented findings are conjectures and not prescriptive or suggestive of causality
    - Need interpretation with more context on how the variables follow from the initiative/program workflows. 


### Data structure 
- There are 1493 CHPs. Each record relates to a CHP.
- Each record is an aggregate of a CHPs activities for the period (90 days until 20-Mar)
- Outliers are excluded in the analysis. Identification of outliers is based on household registration distribution. 


### Limitations
- Data does not have geographic mapping 
- No daily records for time series exploration 



In [ ]:
b4_g = sns.distplot( dataset["Households Registered"])
b4_s = sampleStructure(dataset, "Country") 
b4_d = summarizer( dataset )
## Remove outliers
# updating dataset to HH > 20 and < 250 
dt_outliers = dataset[ (dataset["Households Registered"] < 20) | (dataset["Households Registered"] > 250) ]
dataset.drop( dt_outliers.index, inplace=True) #<< TODO: go back to boolean indexing

a4_g = sns.distplot( dataset["Households Registered"])
a4_s = sampleStructure(dataset, "Country") 
a4_d = summarizer( dataset )

### View of All Data

For this section outliers are included.

- There are 1493 CHPs in the period
    - Average # households per CHP is 100
    - Total of 148,764 households registered 


- Total Sales of KES 646,630
    - Average sales per CHP is kES 433 
    

In [ ]:

print( "All Data Including Outliers - Counts and Averages per CHP")
b4_s

### Summary Stats

For this section outliers are included.


In [ ]:
#dataset[ ["Total Child Assessments", "Total Child Treatments", "All Malaria Assessments","U5 Malaria", "U5 Diarrhea", "U5 Pneumonia", "Pregnancy", "Community Events"]].plot(kind="box", figsize=(17,9), ylim=(-5,35))

b4_d

## Correlations Between Health & Operations

** *From this section onwards outliers are removed.* **


**Interprating Correlations:** Correlation coefficients whose magnitude are
- between 0.9 and 1.0 indicate variables which can be considered very highly correlated.
- between 0.7 and 0.9 indicate variables which can be considered highly correlated.
- between 0.5 and 0.7 indicate variables which can be considered moderately correlated.
- between 0.3 and 0.5 indicate variables which have a low correlation.
- less than 0.3 have little if any (linear) correlation.
Note though, the above may change depending on nature of outcomes/study. E.G. for social sciences 0.3 may be a threshold for strong correlations. 
- TBD: link reference 


### KPIs

**Community events relate more with child assessments than with sales. ** 
- An expectation is that community events are opportunities for driving sales. 
- Comminity events are most associated with child assessments at 0.35. 
- Association with sales is little if at all at 0.17. 



**We potentially rarely discover new families or households outside of those we already service in some way.** 
- New pregnancies are discovered within already known households whose children are attended to OR vice versa 
    - Pregnancy is most correlated with child assessments (0.28) albeit weakly 
    
- Household registration have not particular associations with the other indicators 
    - Correlations are very low for households registered suggesting little associations 
    - The largest magnitude is 0.15 with pregnancy registration.
 
    
**Child care is delivered around community events and pregnancy registration OR vice versa** 
- Child care is associated with community events (0.35) and pregnancy (0.28)
 

PS: - **Households Registered** could indicate reach.  

In [ ]:
#plotCorrz( corrz(dataset, argz_EPMs) )
print("1. Correlation of the key KPIs")
cout = corrz(dataset, argz_EPMs)
cout
plotCorrz( cout )

### Diseases/Cases

**Malaria: we likely know how to do it well and consistently** 
- There are very strong correlations within malaria variables (see first table below).  
- Malaria is most correlated with child treaments compared to the other diseases (see second table below). 
 
**Pneumonia: Most referral activities may be for pneumonia** and potentially b/c of U1s. 
- Correlation values are large betwen diarrhoea and pneumonia than with malaria. 
- Correlation with referral f/u is larger for pneumonia (.26)

In [ ]:
## Malaria  
print( "2. Correlation within Malaria Variables")
cout1 = corrz(dataset, argz_malaria+["U5 Malaria"]  )
cout1

In [ ]:
## All U5 Diseases
## Diseases 
print( "3. Correlation within U5 Diseases")
cout2 = corrz(dataset, argz_U5_diseases+["Total Child Assessments", "Total Child Treatments",  "On Time Referral f/u Treatment U5"])
cout2

plotCorrz( corrz(dataset, argz_U5_diseases+["On Time Referral f/u Treatment U5"]) )

### Sales Activity

- Current sales target is KES 2,040. This is used to derive buckets/categories.
- Majority of CHPs (94%) are below that sales target. 
- For reasonable n within the buckets/categories two categories are used: 1) Meets Target and 2). Misses Target. 


In [ ]:

print( "1. n if we bucket sales in multiples of the target amount of KES 2040")
dataset["Class_Sales_Levels"].value_counts()
print("\n\n")

## Regroup
print("2. n when rebucketing so that sample sizes within buckets are reasonable")
dataset["Class_Sales_Levels"] = np.where( dataset["Sales"] < 2040 , "Below_2040", 
                                         np.where(dataset["Sales"] < 4080, "2040_to_4080",  "Above_4080")) 
dataset["Class_Sales_Levels"].value_counts()
print("\n\n")


In [ ]:
## Regroup 
print( "3. n if bucketing into two groups: Meets Target Vs Misses Target (Occam's Razor + dummy variable)")
dataset["Meets.Sales.Target"] = np.where( dataset["Sales"] < 2040, "Misses Target", "Meets Target")
dataset["Meets.Sales.Target"].value_counts()
print("\n\n")
#pd.get_dummies( dataset["Meets.Sales.Target"])


In [ ]:
print("\n4. Sample Tallies by Sales Target buckets")
sampleStructure(dataset, "Meets.Sales.Target") 
print("\n\n")

** t-tests on the means** 

In [ ]:
from scipy.stats import ttest_ind 

print( "a. t-test Sales means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["Sales"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["Sales"])

print("\n\n")

print( "b. t-test Household registration means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["Households Registered"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["Households Registered"])


print("\n\n")

print( "c. t-test Total child assessments means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["Total Child Assessments"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["Total Child Assessments"])

print("\n\n")

print( "d. t-test all malaria assessments means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["All Malaria Assessments"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["All Malaria Assessments"])


print("\n\n")

print( "e. t-test pregnacy means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["Pregnancy"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["Pregnancy"])


print("\n\n")

print( "f. t-test total U1 assessments means between those that meet sales target and those that don't ")
ttest_ind( dataset[dataset["Meets.Sales.Target"] == "Meets Target"]["Total U1 Assessments"], dataset[dataset["Meets.Sales.Target"] == "Misses Target"]["Total U1 Assessments"])





**SUMMARY**
- Both groups have association between community events and child assessments.

- Both groups have little or no association for sales and other activities

- 




#### Meets Sales Target
- 85 or 6% of the CHPs have met sales target in the period. 


- This group also has higher average household registrations, average pregnancy per CHP and average child assessments per CHP. 
    - t-tests on diferences between those means show statistically significant differences in the means. 
    - They are potentially a representation of strong CHPs.
        - TBD: How does ther performance relate to their graduation scores?

- Ableit small in magnitude, there are negative correlations
    - Household registration has negative association with community events and child assessment. 
    - Sales have negative association with child assessments

- Pregnancy registration are most associated with household registration.
    - Wonder if they register more new households during the year (after graduation) compared to those that don't meet targets.

- Sales have little or no association with the other activities. 

- Community events are associated with child assessments.  
    

In [ ]:
print( "1. Correlations for Meets Sales Targett") 
c1=corrz(dataset[ dataset["Meets.Sales.Target"] == "Meets Target"], argz_EPMs, "spearman") 
c1

print( "1b. Visual for Correlations for Meets Sales Targett")
plotCorrz(c1)

#### Misses Sales Target
- 94% missed sales target of KES 2040.


- This group has similar averages per CHP with the "meet target" group for malaria. 
    - t-tests on diferences between those means show that the groups/means are not statistically significant. They are indeed the similar populations  


- This group has no negative correlations. 


- Households registration is more associated with Sales and pregnancy (0.15: 0.153 and 0.147 respectively). 
    - Could this group be pitching sales during household visits more than the other group? 
    - The magnitudes are low, though 

- Magnitude of sales correlations are larger for this group. However, sales are more associated with child assessment than with community events. 
    - the group that meets sales target has a negative correlation with child assessment. 
    - This group likely makes a sales pitch around each child assessment while the other group doesn't. 
    - **Could it be that pitching during care delivery is not good for closing a sale?** 
    
- Community events are more associated with child assessment for this group as well. 
    

- Pregnancy and child assessment are associated for this group (0.305) just like in the other group, but for this group pregnancy is not as associated with household registration
    - 

In [ ]:
print( "2. Correlations for Misses Sales Targett") 
c1=corrz(dataset[ dataset["Meets.Sales.Target"] == "Misses Target"], argz_EPMs, "spearman") 
c1

print( "2b. Visual for Correlations for Misses Sales Targett")
plotCorrz(c1)

#### Regressions
